You are in a dimly lit game room. There is an old couch and a locked door to the north.
> look around
You see a couch and a locked door.
> examine couch
You find a small key tucked in the cushions.
> take key
You take the key.
> use key
You unlock the door.
> open door
Congratulations! You escape the haunted mansion.


In [186]:
rooms = {
    'main room': { # room 1. Player starts here. 
        'description': "~You wake up on dusty old couch. The room is dimly lit by a candelabra sitting on top of a grand piano. The floor creaks and cobwebs hang from the walls.\n" 
        "And though you can't see anyone else, you're sure you can hear a faint piano melody lingering in the air.~", # Player wakes up on couch and sees room with piano
        'connections': ['bedroom'], # Player must access Bedroom 1 to continue
        'opens with': ["gold key"],
        'items': ["couch", "grand piano", "gold key"] 
    },
    'bedroom': { # room 2, connects with main room and living room
        'description': "~The bedroom's wallpaper is yellow and peeling. Silk drapes a large, oak bedframe in the middle of the room.\n"
        "The kingsized bed is covered in silk sheets and many satin pillows, messy from sleep. Someone was just here. You hear the wind howling outside.\n"
        "You need to find a way out of here.~", # player must search bed to find master key for exit
        'connections': ["main room", 'dining room'], 
        'opens with': ["gold key", "gold key"],
        'items': ["bed", 'silver key']
    },
    'dining room': { # room 3, player can search dining table and/or use key to exit house
        'description': "You're in a cavernous dining room. Darkness envolpes you. The vast chamber echos with sounds unknown.\n" 
        "Your only source is the glow of a long forgotten fire in the room's massive hearth.\n" 
        "You can see a door at the end of the room. You hope it leads outside.",
        'connections': ["bedroom", 'outside'],
        'opens with': ["gold key", "silver key"],
        'items': ["dining table", "fireplace"]
    },
    "outside": { # final challenge. game ends when player exits house
        "description": "You are outside. Fresh air at last!",
        'opens with': ["silver key"],
        "connections": ["dining room"],
    },
    # Additional rooms...
}


In [187]:
class Game:
    def __init__(self, room, item):
        self.location = room
        self.item = item
        self.room_examined = False
        self.has_key = False
        self.inventory = []  # New line
        
    def select_item(self, new_item):
        # this function sets an atribute to a new value
        self.item = new_item

    def examine_room(self):
        self.room_examined = True
        room_dict = rooms[self.location]
        return room_dict
    

    def move_rooms(self, new_location):
        self.location = new_location
        self.room_examined = False
        self.item = None
    
    def print_room(self):
        room_dict = rooms[self.location]
        return room_dict["description"]

    def take_key(self):
        self.has_key = True
    
    def take_item(self, item):  # New method
        self.inventory.append(item)
        print(f"You took the {item}")
    

In [188]:
class Door:
    def __init__(self, is_unlocked: bool, connections: set, opens_with: str):
        self.is_unlocked = is_unlocked 
        self.connections = connections
        self.opens_with = opens_with
        

    def unlock(self, inventory: list): 
        if self.opens_with == "":
            self.is_unlocked = True
        if  self.opens_with in inventory:
            self.is_unlocked = True 
        else:
            print("This door is still locked!")               

    def __repr__(self):
        return (f"Door unlocked: {self.is_unlocked}. Connects with: {self.connections} Opens with: {self.opens_with}") 


In [189]:
player = Game("main room", "couch")

In [190]:
doors_dict = {}
for room in rooms: # loops through each room in rooms dictionary
    doors_dict[room] = [] # creates empty list of rooms in doors dictionary
    print("I am in the room:", room)
    for i, connection in enumerate(rooms[room]["connections"]): # loops through all connections in rooms
        door_exists = False # creating a flag to set the state of doors as not exiting
        if connection in doors_dict: # if connection (aka, one of the connecting rooms)
               for existing_door in doors_dict[connection] : #loop through all its existing connections:
                    if {room, connection} == existing_door.connections: # room = bedroom; existing door = main room; connections = {main room, bedroom}, if it exists --> 
                        doors_dict[room].append(existing_door) # we add all the above info (existing_door) to the doors_dict
                        door_exists = True # and mark the door as existing
                        break # break and loop again
        if not door_exists: # if door doesn't exist -->
            print(f"the {room} connects with: {connection}")
            door1 = Door(False, {room, connection}, opens_with=rooms[room]["opens with"][i]) # --> create a new door
            doors_dict[room].append(door1) # add to door dictionary
print(doors_dict)

I am in the room: main room
the main room connects with: bedroom
I am in the room: bedroom
the bedroom connects with: dining room
I am in the room: dining room
the dining room connects with: outside
I am in the room: outside
{'main room': [Door unlocked: False. Connects with: {'bedroom', 'main room'} Opens with: gold key], 'bedroom': [Door unlocked: False. Connects with: {'bedroom', 'main room'} Opens with: gold key, Door unlocked: False. Connects with: {'bedroom', 'dining room'} Opens with: gold key], 'dining room': [Door unlocked: False. Connects with: {'bedroom', 'dining room'} Opens with: gold key, Door unlocked: False. Connects with: {'outside', 'dining room'} Opens with: silver key], 'outside': [Door unlocked: False. Connects with: {'outside', 'dining room'} Opens with: silver key]}


In [ ]:
# Game loop
first_time = 1
game_running = True
while game_running:
    if first_time: 
        print(f"{rooms[player.location]['description']}.\n", flush = True)
        first_time = 0
    
    # Show available connections
    current_room = rooms[player.location]
    connections = current_room['connections']

    if player.room_examined == True:
        print(f'''What do you want to do?
            Search for items to help you escape: press 1 
            View connecting rooms: press 2  
            Move to another room: press 3
            Inspect an item: press 4
            Type "exit" to exit game  
            ''', flush = True)
    else:
        print(f'''What do you want to do?
            Search for items to help you escape: press 1 
            View connecting rooms: press 2  
            Move to another room: press 3
            Type "exit" to exit game  
            ''', flush = True)   
    
 
    user_input = input("What do you want to do? ")
    
    if user_input == "1":
        room_info = player.examine_room()
        print("You examined the room and found these items:")
          # Get items in the room that the player does not already have
        available_items = [item for item in room_info["items"] if item not in player.inventory]
        if available_items:
            print("You see the following items: " + ", ".join(available_items))
            take_input = input("Type the name of an item to take it, or press Enter to do nothing: ").lower()
            if take_input in available_items:
                player.take_item(take_input) # The take_item method you wrote
            # Now the key will be in player.inventory for the door check
            elif take_input:
                print("That item isn't here.")
        else:
            print("You don't see any other items of interest in this room.")
      
    elif user_input == "2":
       if len(connections) > 0: # turn into a function, 
            print("Rooms you can move to:")
            for i, room in enumerate(connections):
                print(f"    - {room}")
    elif user_input == "3":
        if len(connections) > 0:
            connections_string = " and ".join(connections)
            print(f"What room do you want to move to?\nType one of the following: {connections_string}")
            new_room = input("Your choice: ").lower() # Convert input to lowercase for consistency

            if new_room in connections:
                # Find the specific door that connects the current room to the new room
                door_to_check = None
                for door in doors_dict[player.location]:
                    # A door connects two rooms, so we check if the new_room is one of its connections
                    if new_room in door.connections:
                        # Also ensure this door's other connection is the player's current location
                        if player.location in door.connections:
                            door_to_check = door
                            break # Found the correct door, exit the loop

                if door_to_check:
                    # Attempt to unlock the door with the player's current inventory
                    door_to_check.unlock(player.inventory)

                    # Now, check if the door is unlocked
                    if door_to_check.is_unlocked:
                        # Special case for winning the game
                        if new_room == "outside":
                            print("\nYou use the silver key to unlock the final door and step outside. Fresh air at last!")
                            print("Congratulations! You have escaped!")
                            game_running = False
                        else:
                            print(f"\nYou open the door and move to the {new_room}.")
                            player.move_rooms(new_room)
                            first_time = 1  # This will trigger the new room's description
                    else:
                        # If the door is still locked after the check
                        print(f"\nThe door to the {new_room} is locked. You need the '{door_to_check.opens_with}' to open it.")
                else:
                    print("Error: Could not find the specific door for this connection.") # Should ideally not happen if connections are correctly set up
            else:
                print("\nYou can't go there from here.")
        else:
            print("There are no direct connections from this room.") # Added for clarity if a room has no connections
    elif user_input == "4":
        if player.inventory: # Check if player has any items to inspect
            print(f"Which item do you want to inspect? You have: {', '.join(player.inventory)}")
            item_to_inspect = input("Type the name of an item to inspect: ").lower()

            if item_to_inspect in player.inventory:
                # Provide specific descriptions for important items
                if item_to_inspect == "gold key":
                    print("This is a shiny gold key. It looks like it could open a sturdy, old lock.")
                elif item_to_inspect == "silver key":
                    print("A simple silver key. It might be the final piece to your escape.")
                elif item_to_inspect == "couch": # Example for other items you might have taken (though couch isn't typically 'taken')
                    print("It's a dusty old couch. Not much to see now that you've checked the cushions.")
                # Add more elif for other specific item descriptions as needed
                else:
                    print(f"You inspect the {item_to_inspect}. It seems like a normal {item_to_inspect} and doesn't reveal anything new.")
            else:
                print(f"You don't have a '{item_to_inspect}' in your inventory to inspect.")
        else:
            print("You don't have any items in your inventory to inspect yet. Try searching for some!")

~You wake up on dusty old couch. The room is dimly lit by a candelabra sitting on top of a grand piano. The floor creaks and cobwebs hang from the walls.
And though you can't see anyone else, you're sure you can hear a faint piano melody lingering in the air.~.

What do you want to do?
            Search for items to help you escape: press 1 
            View connecting rooms: press 2  
            Move to another room: press 3
            Type "exit" to exit game  
            
You examined the room and found these items:
You see the following items: couch, grand piano, gold key
That item isn't here.
What do you want to do?
            Search for items to help you escape: press 1 
            View connecting rooms: press 2  
            Move to another room: press 3
            Inspect an item: press 4
            Type "exit" to exit game  
            
You don't have any items in your inventory to inspect yet. Try searching for some!
What do you want to do?
            Search for ite